In [ ]:
#функция, которая проверяет есть ли в графе эйлеров путь, если есть возвращает его
def find_euler_path(graph):
    # Проверка на связность графа
    if not is_connected(graph):
        return None

    # Проверка на количество вершин с нечётной степенью
    odd_degree_nodes = [node for node in graph if len(graph[node]) % 2 != 0]
    if len(odd_degree_nodes) > 2:
        return None

    # Поиск Эйлерова пути
    path = []
    stack = [next(iter(graph))]
    while stack:
        node = stack[-1]
        if graph[node]:
            neighbor = graph[node].pop()
            stack.append(neighbor)
        else:
            path.append(stack.pop())
    
    # Если остались ребра, то Эйлеров путь не существует
    if any(graph[node] for node in graph):
        return None

    # Возвращаем путь в правильном порядке
    return path[::-1]

def is_connected(graph):
    """
    Проверяет, является ли граф связным.
    
    :param graph: Граф в виде словаря
    :return: True, если граф связный, иначе False
    """
    visited = set()
    stack = [next(iter(graph))]
    
    while stack:
        node = stack.pop()
        if node not in visited:
            visited.add(node)
            stack.extend(neighbor for neighbor in graph[node] if neighbor not in visited)
    
    return len(visited) == len(graph)


 G = {
    1: [2, 3],
    2: [1, 3, 4],
    3: [1, 2, 4, 5],
    4: [2, 3, 5],
    5: [3, 4]
}

    # Копия графа для работы алгоритма
graph_copy = {node: neighbors[:] for node, neighbors in G.items()}

    # Поиск Эйлерова пути
euler_path = find_euler_path(graph_copy)
    
if euler_path:
    print("Эйлеров путь:", euler_path)
else:
    print("Эйлеров путь не существует.")

In [ ]:
#алгоритм Джонсона
def bellman_ford(graph, source):
    """
    Алгоритм Беллмана-Форда для вычисления потенциалов.
    
    :param graph: Граф в виде {вершина: [(сосед, вес), ...]}
    :param source: Стартовая вершина
    :return: Словарь расстояний или None при наличии отрицательного цикла
    """
    distance = {node: float('inf') for node in graph}
    distance[source] = 0

    for _ in range(len(graph) - 1):
        for node in graph:
            for neighbor, weight in graph[node]:
                if distance[node] + weight < distance[neighbor]:
                    distance[neighbor] = distance[node] + weight

    # Проверка на отрицательные циклы
    for node in graph:
        for neighbor, weight in graph[node]:
            if distance[node] + weight < distance[neighbor]:
                return None  
    return distance

def dijkstra(graph, start):
    """
    Алгоритм Дейкстры для поиска кратчайших путей.
    
    :param graph: Граф в виде {вершина: [(сосед, вес), ...]}
    :param start: Стартовая вершина
    :return: Словарь кратчайших расстояний
    """
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    visited = set()

    while len(visited) < len(graph):
        # Находим вершину с минимальным расстоянием
        current = None
        min_dist = float('inf')
        for node in graph:
            if node not in visited and distances[node] < min_dist:
                min_dist = distances[node]
                current = node
        if current is None:
            break

        visited.add(current)

        # Обновляем расстояния до соседей
        for neighbor, weight in graph[current]:
            new_dist = distances[current] + weight
            if new_dist < distances[neighbor]:
                distances[neighbor] = new_dist

    return distances

def johnson(graph):
    """
    Алгоритм Джонсона для всех пар кратчайших путей.
    
    :param graph: Ориентированный граф с весами (возможны отрицательные)
    :return: Матрица расстояний или None при отрицательных циклах
    """
    if not graph:
        return {}

    # Добавляем новую вершину 'Q' с нулевыми ребрами
    new_graph = {node: [] for node in graph}
    new_graph['Q'] = []
    for node in graph:
        new_graph[node] = graph[node].copy()
        new_graph['Q'].append((node, 0))

    # Вычисляем потенциалы
    h = bellman_ford(new_graph, 'Q')
    if h is None:
        return None  

    # Удаляем временную вершину
    del new_graph['Q']
    del h['Q']

    # Перевзвешиваем ребра
    reweighted_graph = {node: [] for node in graph}
    for node in graph:
        for neighbor, weight in graph[node]:
            new_weight = weight + h[node] - h[neighbor]
            reweighted_graph[node].append((neighbor, new_weight))

    # Запускаем Дейкстру для каждой вершины
    all_distances = {}
    for node in graph:
        dist = dijkstra(reweighted_graph, node)
        # Восстанавливаем исходные расстояния
        all_distances[node] = {n: dist[n] + h[n] - h[node] for n in dist}

    return all_distances

graph = {
    'A': [('B', -1), ('C', 4)],
    'B': [('C', 3), ('D', 2), ('E', 2)],
    'C': [],
    'D': [('B', 1), ('C', 5)],
    'E': [('D', -3)]
}

result = johnson(graph)
    
if result:
    print("Кратчайшие расстояния:")
    for source in result:
        print(f"Из {source}:")
        for target in result[source]:
            print(f"  → {target}: {result[source][target]}")
else:
    print("Граф содержит отрицательный цикл")


In [ ]:
#Задача
def can_get_infinitely_rich(exchanges, start='RUB'):
    """
    Определяет возможность бесконечного обогащения через цепочку обменников.
    
    :param exchanges: Список кортежей вида (исходная_валюта, целевая_валюта, курс)
    :param start: Начальная валюта (по умолчанию 'RUB')
    :return: True если можно бесконечно увеличивать капитал, иначе False
    """
    # Строим граф и собираем все валюты
    graph = {}
    currencies = set()
    for u, v, r in exchanges:
        currencies.add(u)
        currencies.add(v)
        if u not in graph:
            graph[u] = []
        graph[u].append((v, r))
    
    n = len(currencies)
    if n == 0 or start not in currencies:
        return False
    
    # Инициализация: только стартовая валюта имеет ненулевой баланс
    balance = {curr: 0.0 for curr in currencies}
    balance[start] = 1.0
    
    # Алгоритм Беллмана-Форда (n-1 итераций)
    for _ in range(n - 1):
        updated = False
        for u in graph:
            if balance[u] == 0:
                continue
            for (v, rate) in graph[u]:
                new_balance = balance[u] * rate
                if new_balance > balance[v]:
                    balance[v] = new_balance
                    updated = True
        if not updated:
            break
    
    # Поиск вершин, участвующих в положительных циклах
    profit_nodes = set()
    for u in graph:
        if balance[u] == 0:
            continue
        for (v, rate) in graph[u]:
            if balance[u] * rate > balance[v]:
                profit_nodes.add(v)
    
    if not profit_nodes:
        return False
    
    # Проверка достижимости стартом цикла через BFS
    visited = set()
    queue = [start]
    while queue:
        curr = queue.pop(0)
        if curr in visited:
            continue
        visited.add(curr)
        if curr in profit_nodes:
            return True
        if curr in graph:
            for (neighbor, _) in graph[curr]:
                if neighbor not in visited:
                    queue.append(neighbor)
    
    return False
